# Part 3: Scaling Up — The Tetrahedral Group $T_d$

**6.7970/8.750 Symmetry and its Application to Machine Learning**

In Parts 1–2 we worked with $D_4$ (order 8, 5 irreps). Now we apply the same tools to a **larger** group that arises constantly in chemistry and physics: the **tetrahedral group** $T_d$ (order 24, 5 irreps). By the end you will:

1. Build $T_d$ from its 3D symmetry operations using `pymatgen`
2. Find all irreps and label them by matching to the standard character table
3. Decompose the 3D **natural representation** (the matrices themselves)
4. Compute selected tensor products and compare to the reference

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/decomposition_lecture/03_larger_groups_Td.ipynb)

## Setup

In [ ]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip
!pip install pymatgen

In [ ]:
import numpy as np
from symm4ml import groups, linalg, rep, vis
from symm4ml.utils import match_character_tables, print_character_table, print_multiplication_table
from pymatgen.symmetry.groups import PointGroup

---
## 1. Building the $T_d$ group

$T_d$ is the **full symmetry group of a tetrahedron**: 24 operations including rotations, improper rotations, and mirrors.

Rather than writing out generators by hand, we use `pymatgen` to look up all symmetry operations from the [Hermann–Mauguin notation](https://en.wikipedia.org/wiki/Hermann%E2%80%93Mauguin_notation) $\bar{4}3m$.

In [ ]:
# Build Td from pymatgen: Schoenflies Td = Hermann-Mauguin -43m
Td_group = PointGroup('-43m')

# Extract the 3×3 rotation/improper-rotation matrices
Td = np.stack([op.rotation_matrix for op in Td_group.symmetry_ops], axis=0)

Td_table = groups.make_multiplication_table(Td)
h = len(Td_table)

print(f"Td has {h} elements (3D matrices of shape {Td[0].shape})")
print(f"Conjugacy classes: {len(groups.conjugacy_classes(Td_table))}")

In [ ]:
# Visualize the 24 matrices
vis.plot_matrices(Td, figsize=(20, 3));

Unlike $D_4$ (all permutation matrices in the regular rep), these are 3×3 matrices with entries from $\{-1, 0, 1\}$. They include:
- The identity
- 8 rotations by $\pm 120°$ around the four body diagonals ($C_3$)
- 3 rotations by $180°$ ($C_2$)
- 6 improper rotations ($S_4$)
- 6 mirror reflections ($\sigma_d$)

---
## 2. Finding the irreps

The decomposition algorithm from Part 1 works identically here. One subtlety: `infer_irreps` uses random matrices internally, and some random seeds can produce **complex-valued** irreps (which are equivalent but less convenient). We search for a seed that gives **real** irreps.

In [ ]:
# Find a random seed that gives real-valued irreps
# for seed in range(100):
for seed in [91]:  # 91 worked for us
    np.random.seed(seed)
    Td_irreps_raw = rep.infer_irreps(Td_table)
    if all(ir.dtype == np.float64 for ir in Td_irreps_raw):
        print(f"Found real irreps with seed = {seed}")
        break

print(f"\nFound {len(Td_irreps_raw)} irreps:")
for i, ir in enumerate(Td_irreps_raw):
    print(f"  Irrep {i}: dim {ir.shape[1]}")

dims = [ir.shape[1] for ir in Td_irreps_raw]
print(f"\nDimension check: {' + '.join(f'{d}²' for d in dims)} = {sum(d**2 for d in dims)} = h = {h}  \u2713")

We get 5 irreps with dimensions $1, 1, 2, 3, 3$, satisfying $1^2 + 1^2 + 2^2 + 3^2 + 3^2 = 24$.

---
## 3. Labeling the irreps

The [standard character table for $T_d$](http://gernot-katzers-spice-pages.com/character_tables/Td.html) labels the irreps as:

| | $E$ | $8C_3$ | $3C_2$ | $6S_4$ | $6\sigma_d$ |
|---|---|---|---|---|---|
| $A_1$ | 1 | 1 | 1 | 1 | 1 |
| $A_2$ | 1 | 1 | 1 | −1 | −1 |
| $E$ | 2 | −1 | 2 | 0 | 0 |
| $T_1$ | 3 | 0 | −1 | 1 | −1 |
| $T_2$ | 3 | 0 | −1 | −1 | 1 |

We use the same character-matching procedure from Part 1.

In [ ]:
conj_classes = groups.conjugacy_classes(Td_table)
conj_list = sorted(conj_classes, key=lambda cc: (len(cc), min(cc)))

print(f"{len(conj_list)} conjugacy classes:\n")
for cc in conj_list:
    print(f"  size {len(cc):>2d}: elements {set(cc)}")

In [ ]:
# Compute our character table
char_table = rep.character_table(Td_irreps_raw, conj_list)

print("Character table (unordered):\n")
print(np.round(char_table.real, 2))

In [ ]:
# Reference character table for Td (standard convention)
# Columns: E, 8C3, 3C2, 6S4, 6σd
names = ['A₁', 'A₂', 'E', 'T₁', 'T₂']
class_names = ['E', '8C₃', '3C₂', '6S₄', '6σ_d']
ref_chars = np.array([
    [1,  1,  1,  1,  1],   # A1
    [1,  1,  1, -1, -1],   # A2
    [2, -1,  2,  0,  0],   # E
    [3,  0, -1,  1, -1],   # T1
    [3,  0, -1, -1,  1],   # T2
])

# Compute character table in whatever order we have
conj_list = list(conj_classes)
char_table = rep.character_table(Td_irreps_raw, conj_list)

# Find the row (irrep) and column (class) permutations
row_perm, col_perm = match_character_tables(char_table, ref_chars)

irreps = [Td_irreps_raw[i] for i in row_perm]
conj_std = [conj_list[j] for j in col_perm]

print("Matching our irreps to standard T_d labels:")
for std_idx, our_idx in enumerate(row_perm):
    print(f"  Td_irreps[{our_idx}] → {names[std_idx]} (dim {irreps[std_idx].shape[1]})")

In [ ]:
# Print the character table in standard order
char_table_std = rep.character_table(irreps, conj_std)
print_character_table(char_table_std, names, class_names,
                      title="Character table of T_d (standard ordering)")

---
## 4. The 3D natural representation

The 3×3 matrices we started with **are themselves a representation** of $T_d$. This is the **natural representation** (sometimes called the *vector* or *defining* representation). Is it irreducible?

In [ ]:
print(f"Is the 3D natural rep a valid representation? {rep.is_a_representation(Td_table, Td)}")
print(f"Is it irreducible? {rep.is_an_irrep(Td_table, Td)}")

# Which irrep is it?
for name, ir in zip(names, irreps):
    if ir.shape[1] == 3 and rep.are_isomorphic(Td, ir):
        print(f"\nThe natural rep is isomorphic to {name}!")
        break

The 3D natural representation is already irreducible and corresponds to $T_2$ — the 3D irrep under which ordinary 3D vectors transform.

---
## 5. Decomposing a reducible representation

Let's build a reducible representation and decompose it. A natural choice: the **permutation representation** of $T_d$ acting on the 4 vertices of a tetrahedron.

In [ ]:
# Tetrahedron vertices (inscribed in a cube)
tetrahedron = np.array([
    [-1., -1.,  1.],
    [-1.,  1., -1.],
    [ 1., -1., -1.],
    [ 1.,  1.,  1.],
])

# Build the 4D permutation representation:
# For each group element g, find which vertex maps to which
n_verts = len(tetrahedron)
perm_rep = np.zeros((h, n_verts, n_verts))

for g_idx in range(h):
    rotated = (Td[g_idx] @ tetrahedron.T).T  # apply g to all vertices
    for i in range(n_verts):
        for j in range(n_verts):
            if np.allclose(rotated[i], tetrahedron[j], atol=1e-8):
                perm_rep[g_idx, j, i] = 1.0

print(f"Permutation rep shape: {perm_rep.shape}")
print(f"Is a valid representation? {rep.is_a_representation(Td_table, perm_rep)}")
print(f"Is irreducible? {rep.is_an_irrep(Td_table, perm_rep)}")

In [ ]:
# Decompose it
sub_irreps = rep.decompose_rep_into_irreps(perm_rep)

print(f"The 4D perm rep decomposes into {len(sub_irreps)} irreps:\n")
for k, sub in enumerate(sub_irreps):
    label = '?'
    for name, ir in zip(names, irreps):
        if rep.are_isomorphic(sub, ir):
            label = name
            break
    print(f"  Component {k+1}: dim {sub.shape[1]}  \u2192  {label}")

dims_sub = [s.shape[1] for s in sub_irreps]
print(f"\nDimension check: {' + '.join(map(str, dims_sub))} = {sum(dims_sub)} = {perm_rep.shape[1]}  \u2713")

The 4D permutation representation decomposes as $A_1 \oplus T_2$. This makes physical sense:
- $A_1$ is the trivial irrep — it corresponds to the "center of mass" mode (all vertices move together)
- $T_2$ is the 3D irrep — it captures the three independent ways the tetrahedron can deform

---
## 6. Tensor products of $T_d$ irreps

Let's compute a few interesting tensor products.

In [ ]:
def decompose_and_label(tp_rep, names, irreps):
    """Decompose a rep and return labels of its irreducible components."""
    sub = rep.decompose_rep_into_irreps(tp_rep)
    labels = []
    for s in sub:
        for name, ir in zip(names, irreps):
            if rep.are_isomorphic(s, ir):
                labels.append(name)
                break
        else:
            labels.append('?')
    return labels

# T2 ⊗ T2 (vectors ⊗ vectors — this gives rank-2 tensors!)
T2 = irreps[4]  # the 3D irrep
T2_x_T2 = rep.tensor_product(T2, T2)
labels = decompose_and_label(T2_x_T2, names, irreps)

print(f"T\u2082 \u2297 T\u2082 (dim {T2_x_T2.shape[1]}) = {' \u2295 '.join(sorted(labels))}")
print(f"\nThis tells us how rank-2 tensors decompose under T_d symmetry!")

In [ ]:
# E ⊗ T2
E_irrep = irreps[2]  # the 2D irrep
E_x_T2 = rep.tensor_product(E_irrep, T2)
labels = decompose_and_label(E_x_T2, names, irreps)
print(f"E \u2297 T\u2082 (dim {E_x_T2.shape[1]}) = {' \u2295 '.join(sorted(labels))}")

# E ⊗ E
E_x_E = rep.tensor_product(E_irrep, E_irrep)
labels = decompose_and_label(E_x_E, names, irreps)
print(f"E \u2297 E   (dim {E_x_E.shape[1]}) = {' \u2295 '.join(sorted(labels))}")

---
## 7. Full multiplication table for $T_d$

Let's build the complete table, just as we did for $D_4$ in Part 2.

In [ ]:
# Compute the full irrep multiplication table
dp_table = rep.direct_product_table(irreps, names)

# Format for printing
mul_table = [['⊕'.join(sorted(dp_table[i, j])) for j in range(len(names))]
             for i in range(len(names))]

print_multiplication_table(mul_table, names, title="Irrep multiplication table for T_d")

### Checking with characters

We can verify the entire table using the character shortcut (much faster for large groups):

In [ ]:
chi = rep.character_table(irreps, conj_std)
class_sizes = np.array([len(cc) for cc in conj_std])

# Build the same table using only characters
mul_table_chi = []
for i in range(len(names)):
    row = []
    for j in range(len(names)):
        chi_prod = chi[i] * chi[j]
        labels = []
        for k in range(len(names)):
            n_k = int(round(np.sum(class_sizes * np.conj(chi[k]) * chi_prod).real / h))
            labels.extend([names[k]] * n_k)
        row.append('⊕'.join(sorted(labels)))
    mul_table_chi.append(row)

print_multiplication_table(mul_table_chi, names, title="Multiplication table via characters")
print("\n(Should match the table above!)")

---
## 8. Comparison: $D_4$ vs $T_d$

| Property | $D_4$ | $T_d$ |
|---|---|---|
| **Order** $h$ | 8 | 24 |
| **Conjugacy classes** | 5 | 5 |
| **Irrep dimensions** | 1, 1, 1, 1, 2 | 1, 1, 2, 3, 3 |
| **Dimension formula** | $4(1^2) + 2^2 = 8$ | $2(1^2) + 2^2 + 2(3^2) = 24$ |
| **Largest irrep** | 2D ($E$) | 3D ($T_1, T_2$) |
| **Physical setting** | Square symmetry | Tetrahedron, $sp^3$ orbitals |

Despite being 3× larger, $T_d$ has the **same number of irreps** as $D_4$! The number of irreps equals the number of conjugacy classes, not the order of the group.

The algorithms scale well: `infer_irreps` works on the $24 \times 24$ regular representation with no trouble. For even larger groups, the character shortcut avoids constructing explicit matrices entirely.

---
## Summary

Over these three notebooks we have:

1. **Part 1**: Built the decomposition algorithm from scratch (commutant → random combination → eigenspaces) and found all irreps of $D_4$
2. **Part 2**: Computed tensor products, decomposed them into irreps, and built the full multiplication table
3. **Part 3**: Applied the same tools to $T_d$, showing the algorithms scale to physically relevant groups

These tools are the computational backbone of **equivariant machine learning**: tensor products of irreps determine which linear maps are allowed between feature spaces, and the Clebsch–Gordan decomposition tells us how to parametrize them.